# Retrieve the equations from the algorithm.
In this Notebook it is investigated how if the original equations can be retrieved for the Lorenz Model.

In [ ]:
from scipy.integrate import odeint
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy as sp
import random
import importlib
from tqdm import tqdm

import matplotlib.pyplot as plt
from kalman_reconstruction.kalman import (
    Kalman_SEM,
    Kalman_smoother,
)
from kalman_reconstruction.kalman_time_dependent import (
    Kalman_SEM_time_dependent,
    Kalman_smoother_time_dependent,
)
from kalman_reconstruction.statistics import gaussian_kernel_1D

In [ ]:
plt.rcParams["figure.figsize"] = (10, 5)
# Set default matplotlib style
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20
plt.style.use("seaborn-v0_8-whitegrid")
# plt.style.use('dark_background')

plt.rcParams["figure.figsize"] = (10.0, 6.0)
plt.rc("font", size=MEDIUM_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc("legend", loc="upper right")
plt.rc("lines", linewidth=1.5)

# use colorblind save colors https://davidmathlogic.com/colorblind
colors = [
    "#CC6677",
    "#6E9CB3",
    "#AA4499",
    "#CA8727",
    "#A494F5",
    "#44AA99",
    "#D6BE49",
]
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)

## Kalman SEM timedependent

### Important parameters

In [ ]:
# index of the unobserved component
i_unobs_comp = 0

# integration time step
dt = 0.01

# variance of the random white noise of z
variance_unobs_comp = 5

# variance of the observation error used in Kalman
variance_obs_comp = 0.0001

# number of Lorenz-63 times
nb_loop = 10

# number of SEM iterations
nb_iter_SEM = 10

### Generate simulated data

In [ ]:
# Lorenz-63 dynamical model
def Lorenz_63(x, dx, sigma, rho, beta):
    dx = np.empty((3))
    dx[0] = sigma * (x[1] - x[0])
    dx[1] = x[0] * (rho - x[2]) - x[1]
    dx[2] = x[0] * x[1] - beta * x[2]
    return dx


# Lorenz-63 parameters
x0 = np.array([8, 0, 30])  # initial condition
sigma = 10
rho = 28
beta = 8 / 3  # physical parameters

# time and truth
t = np.arange(0.01, nb_loop, dt)
x_t = odeint(Lorenz_63, x0, np.arange(0.01, nb_loop, dt), args=(sigma, rho, beta))

# observations
y_t = x_t.copy()

## Time independent reconstruction

### All components observed.
$x = [x_1, x_2, x_3]$

In [ ]:
y = y_t.copy()
# state
x = y.copy()

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

# colors and labels of the components
tab_labels = ["$x_1$", "$x_2$", "$x_3$"]

# plot the components
plt.plot(t, x_t, label=tab_labels)
plt.xlabel("System loops")
plt.ylabel("Components of the system")
plt.legend(loc=1)
plt.title("All Components")

#### Calculate the coefficients from the model.
Note that the equation $\frac{d\bar{x}}{dt} = M \cdot \bar{x}$ can not solve the problem, because it is a non-linear PDE.

In [ ]:
# regression coefficients
dx_dt = np.diff(x_t, axis=0) / dt
regress = np.linalg.lstsq(dx_dt, x_t[0:-1], rcond=None)
regress_coef_true = regress[0]
x_regress = dx_dt @ regress_coef_true

In [ ]:
fig, ax0 = plt.subplots(1, 1)
ax0.plot(t[0:-1], x_t[0:-1], label=[tab + " true state" for tab in tab_labels])
ax0.legend(loc="lower left")
ax1 = ax0.twinx()
ax1.sharey(ax0)
ax1.plot(
    t[0:-1],
    x_regress - x_regress[0] + x_t[0],
    label=[tab + " lin. matrix" for tab in tab_labels],
    linestyle="--",
)
ax1.legend(loc="lower center")
ax0.set_xlabel("System loops")
ax0.set_ylabel("Components of the system")
ax0.set_title("True state vs linear matrix equation.");

In [ ]:
def coef_to_ode_str(coef):
    eq_dim = np.shape(coef)[0]
    state_dim = np.shape(coef)[1]
    state_names = [f"x{num}" for num in range(state_dim)]
    for i in range(eq_dim):
        res = f"{state_names[i]}"
        for j in range(state_dim):
            res += f"\t{coef[i][j]:.3f} d{state_names[j]}/dt +"
        print(res)


coef_to_ode_str(regress_coef_true)

### V0: $x = [x_2, x_3]$

In [ ]:
# state
y = y_t[:, [1, 2]]
x = np.c_[y[:, 0], y[:, 1]]

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

# colors and labels of the components
tab_labels = ["$x_2$", "$x_3$"]

# plot the components
plt.plot(t, x)
plt.xlabel("System loops")
plt.ylabel("Components of the system")
plt.legend(tab_labels, loc=1)
plt.title("Observed components")

In [ ]:
# kalman parameters
H = np.eye(n)
R = variance_obs_comp * np.eye(p)

# stochastic EM
# x_s_V0, P_s_V0, M_V0, loglik_V0, x, x_f_V0, Q_V0 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

x_s_V0, P_s_V0, M_V0, loglik_V0, x, x_f_V0, Q_V0 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

### V1: $x = [x_2, x_3, z_1]$

In [ ]:
# state
y = y_t[:, [1, 2]]
z = np.random.normal(loc=0, scale=variance_unobs_comp, size=np.shape(y)[0])
x = np.c_[y[:, 0], y[:, 1], z]

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

In [ ]:
##################
### FIGURE 1-a ###
##################

tab_labels = [
    "$y_2 \ (2^{nd} \ \mathrm{Lorenz \ component})$",
    "$y_3 \ (3^{rd} \ \mathrm{Lorenz \ component})$",
    "$z_1 = Random(\mathcal{N}(0,\sigma^2))$",
]
for i in [2, 0, 1]:
    plt.plot(t, x[:, i], color=colors[i], label=tab_labels[i])
plt.xlabel("Time")
plt.legend(loc=1)
plt.ylim([-25, 45])
plt.xlim([t[0], t[-1]])

**NOTE** The function Kalman_SEM mutates the input array, is that wanted?

In [ ]:
# kalman parameters
H = np.delete(np.eye(n), 2, axis=0)
R = variance_obs_comp * np.eye(p)

# stochastic EM
x_s_V1, P_s_V1, M_V1, loglik_V1, x, x_f_V1, Q_V1 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

# Because Kalman filter mutates input array x, store it in x_V1
x_V1 = x.copy()

In [ ]:
# ##################
# ### FIGURE 1-b ###
# ##################

# # tab_labels = ['$x^s_2$', '$x^s_3$', '$x^s_1 \pm 1.96 \sqrt{P^s_1}$']
# for i in [2, 0, 1]:
#     plt.plot(t, x_s_V1[:, i], color=colors[i], label=tab_labels[i])
#     plt.fill_between(
#         t,
#         x_s_V1[:, i] - 1.96 * np.sqrt(P_s_V1[:, i, i]),
#         x_s_V1[:, i] + 1.96 * np.sqrt(P_s_V1[:, i, i]),
#         facecolor=colors[i],
#         alpha=0.30,
#     )

# plt.xlabel("Time")
# plt.legend(loc=1)
# plt.ylim([-25, 45])
# plt.xlim([t[0], t[-1]])
# # plt.savefig("timedependent_model_first_draft.png")

#### Identify the Coefficients:
$z_1 = a_1 \dot{x_1} + a_2 \dot{x_2} $ 

In [ ]:
# regression coefficients
# Calculate the 1st order time derivative
dx_dt_V1 = np.diff(x_s_V1, axis=0) / dt

# perform a linear approximation for the whole state vector
regress_V1 = np.linalg.lstsq(dx_dt_V1, x_s_V1[0:-1], rcond=None)
regress_coef_V1 = regress_V1[0]
x_regress_V1 = dx_dt_V1 @ regress_coef_V1
# perform a linear approximation for
# d/dt(x2,x3) * a = z1
regress_a = np.linalg.lstsq(dx_dt_V1[:, [0, 1]], x_s_V1[0:-1, -1], rcond=None)
regress_a_coef = regress_a[0]
z1_V1 = regress_a_coef[0] * dx_dt_V1[:, 0] + regress_a_coef[1] * dx_dt_V1[:, 1]
regress_a

In [ ]:
fig, ax0 = plt.subplots(1, 1)
for i in [0, 1, 2]:
    ax0.plot(
        t[0:-1], x_s_V1[0:-1, i], label=tab_labels[i] + " true state", color=colors[i]
    )
    plt.fill_between(
        t,
        x_s_V1[:, i] - 1.96 * np.sqrt(P_s_V1[:, i, i]),
        x_s_V1[:, i] + 1.96 * np.sqrt(P_s_V1[:, i, i]),
        facecolor=colors[i],
        alpha=0.30,
        zorder=10,
    )
for i in [2]:
    ax0.plot(
        t[0:-1],
        (x_regress_V1 - x_regress_V1[0] + x_s_V1[0])[:, i],
        label=tab_labels[i] + " full matrix",
        color=colors[i + 1],
        linewidth=3,
    )
ax0.plot(
    t[0:-1], z1_V1, label=tab_labels[i] + " eq. 6a", color=colors[i + 2], linewidth=3
)
ax0.legend(loc="lower left")
ax0.set_xlabel("System loops")
ax0.set_ylabel("Components of the system")
ax0.set_title("True state vs linear matrix equation.");

### V2: $x = [x_2, x_3, z_1, z_2]$

In [ ]:
# state
z = np.random.normal(
    loc=x_t[:, i_unobs_comp] * 0, scale=variance_unobs_comp, size=np.shape(y)[0]
)
x = np.c_[x_V1, z]

# shapes
n = np.shape(x)[1]
p = np.shape(y)[1]

# colors and labels of the components
tab_colors = ["C0", "C1", "C2", "C3"]
tab_labels = ["$x_2$", "$x_3$", "$z_1$", "$z_2$"]

# plot the components
plt.plot(t, x, label=tab_labels)
plt.xlabel("System loops")
plt.ylabel("Components of the system")
plt.legend(loc=1)
plt.title("Initial x")

In [ ]:
# kalman parameters
H = np.delete(np.eye(n), [2, 3], axis=0)
R = variance_obs_comp * np.eye(p)

# stochastic EM
x_s_V2, P_s_V2, M_V3, loglik_V2, x, x_f_V2, Q_V2 = Kalman_SEM(x, y, H, R, nb_iter_SEM)

#### Identify the Coefficients:
$z_2 = b_1 \dot{z_1} + b_2 \dot{x_2} + b_3 \dot{x_3} $ 

In [ ]:
# regression coefficients
# Calculate the 1st order time derivative
dx_dt_V2 = np.diff(x_s_V2, axis=0) / dt

# perform a linear approximation for the whole state vector
regress_V2 = np.linalg.lstsq(dx_dt_V2, x_s_V2[0:-1], rcond=None)
regress_coef_V2 = regress_V2[0]
x_regress_V2 = dx_dt_V2 @ regress_coef_V2
# perform a linear approximation for
# d/dt(x2,x3) * a = (z1)
regress_a_V2 = np.linalg.lstsq(dx_dt_V2[:, [0, 1]], x_s_V2[0:-1, [-2]], rcond=None)
regress_a_V2_coef = regress_a_V2[0]
z1_V2 = regress_a_V2_coef[0] * dx_dt_V2[:, 0] + regress_a_V2_coef[1] * dx_dt_V2[:, 1]
# perform a linear approximation for
# d/dt(x2,x3) * b = (z1,z2)
regress_b = np.linalg.lstsq(dx_dt_V2[:, [0, 1, 2]], x_s_V2[0:-1, [-1]], rcond=None)
regress_b_coef = regress_b[0]
z2_V2 = (
    regress_b_coef[0] * dx_dt_V2[:, 0]
    + regress_b_coef[1] * dx_dt_V2[:, 1]
    + regress_b_coef[2] * dx_dt_V2[:, 2]
)
regress_b

In [ ]:
fig, ax0 = plt.subplots(1, 1)
for i in [0, 1, 2, 3]:
    ax0.plot(
        t[0:-1],
        x_s_V2[0:-1, i],
        label=tab_labels[i] + " true state",
        color=colors[i],
        linestyle=":",
    )
    plt.fill_between(
        t,
        x_s_V2[:, i] - 1.96 * np.sqrt(P_s_V2[:, i, i]),
        x_s_V2[:, i] + 1.96 * np.sqrt(P_s_V2[:, i, i]),
        facecolor=colors[i],
        alpha=0.30,
        zorder=10,
    )
for i in [2, 3]:
    ax0.plot(
        t[0:-1],
        (x_regress_V2 - x_regress_V2[0])[:, i],
        label=tab_labels[i] + "full matrix",
        color=colors[i],
        linestyle="--",
    )
    if i == 2:
        ax0.plot(
            t[0:-1],
            z1_V2,
            label=tab_labels[i] + " eq. 6a",
            color=colors[i],
            linestyle="-",
        )
    if i == 3:
        ax0.plot(
            t[0:-1],
            z2_V2,
            label=tab_labels[i] + " eq. 6b",
            color=colors[i],
            linestyle="-",
        )

ax0.legend(loc="lower left")
ax0.set_xlabel("System loops")
ax0.set_ylabel("Components of the system")
ax0.set_title("True state vs full matrix and eq.6");

**It is  still known that the true state is related to the PDE**

In [ ]:
def calculate_x1(x2, x3, z1, z2, a, b, sigma, beta, rho):
    """Calculate the missing component x1 using
    - x2 of the Kalman_SEM
    - x3 of the Kalman_SEM
    - z1 of the Kalman_SEM
    - z2 of the Kalman_SEM
    - a containing the coefficients a1, a2, a3 (a1 is not used)
    - b containing the coefficients b1, b2, b3
    For a and b refer to eq.6(a+b) in the paper Tandeo et. al 2023.

    Parameters
    ----------
    x2: np.ndarray
        x2 of the Lorenz model.
        Needs to be dimension 1 with length N.
    x3: np.ndarray
        x3 of the Lorenz model.
        Needs to be dimension 1 with length N.
    z1: np.ndarray
        1st additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    z2: np.ndarray
        2nd additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    a: np.ndarray
        Array containing the coefficients [a2,a3] for the equation:
        z1 = a2* d/dt(x2) + a3* d/dt(x3)
    b: np.ndarray
        Array containing the coefficients [b1, b2, b3] for the equation:
        z2 = b1* d/dt(z1) + b2* d2/dt2(x3) + b3* d2/dt2(x3)

    Returns
    -------
    np.ndarray
        The estimation of x1 based the coefficients provided and the assumption of knowing
        d/dt(x2) = ... and
        d/dt(x3) = ...
        from the Lorenz Model PDE.

    """
    a2 = a[0]
    a3 = a[1]
    b1 = b[0]
    b2 = b[1]
    b3 = b[2]
    # calculate the nominator
    nominator = z1 + a2 * x2 + a3 * beta * x3
    denominator = a2 * (rho - x3) + a3 * x2
    return nominator / denominator


def calculate_dx1dt(x2, x3, z1, z2, a, b, sigma, beta, rho):
    """Calculate the 1st order time derivative of missing component x1 using
    - x2 of the Kalman_SEM
    - x3 of the Kalman_SEM
    - z1 of the Kalman_SEM
    - z2 of the Kalman_SEM
    - a containing the coefficients a1, a2, a3 (a1 is not used)
    - b containing the coefficients b1, b2, b3
    For a and b refer to eq.6(a+b) in the paper Tandeo et. al 2023.

    Parameters
    ----------
    x2: np.ndarray
        x2 of the Lorenz model.
        Needs to be dimension 1 with length N.
    x3: np.ndarray
        x3 of the Lorenz model.
        Needs to be dimension 1 with length N.
    z1: np.ndarray
        1st additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    z2: np.ndarray
        2nd additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    a: np.ndarray
        Array containing the coefficients [a2,a3] for the equation:
        z1 = a2* d/dt(x2) + a3* d/dt(x3)
    b: np.ndarray
        Array containing the coefficients [b1, b2, b3] for the equation:
        z2 = b1* d/dt(z1) + b2* d2/dt2(x3) + b3* d2/dt2(x3)

    Returns
    -------
    np.ndarray
        The estimation of x1 based the coefficients provided and the assumption of knowing
        d/dt(x2) = ... and
        d/dt(x3) = ...
        from the Lorenz Model PDE.

    """
    a2 = a[0]
    a3 = a[1]
    b1 = b[0]
    b2 = b[1]
    b3 = b[2]

    x1 = calculate_x1(x2, x3, z1, z2, a, b, sigma, beta, rho)
    # calculate the nominator
    coef_1 = x1 * (
        -b1 * a2 * (x1 * x2 - beta * x3)
        + b2 * a3 * (x1 * (rho - x3) - x2)
        + b2 * (rho - x3)
        + b3 * x3
    )
    coef_2 = (
        -b1 * a2 * (x1 * (rho - x3 - x2))
        + -b2 * a3 * (x1 * x2 - beta * x3)
        + -b2 * x2
        + -beta * b2 * x3
    )
    nominator = z2 - coef_1 - coef_2
    denominator = b1 * a2 * (rho - x3) + b2 * a3 * x2
    return nominator / denominator


x1_restored = calculate_x1(
    x2=x_s_V2[:, 0],
    x3=x_s_V2[:, 1],
    z1=x_s_V2[:, 2],
    z2=x_s_V2[:, 3],
    a=regress_a_V2_coef,
    b=regress_b_coef,
    sigma=sigma,
    beta=beta,
    rho=rho,
)

fig, ax0 = plt.subplots(1, 1)
ax0.plot(t, x_t[:, 0], label=r"$x_1^{true}$", alpha=1, lw=2)
ax0.plot(t, x1_restored, label=r"$x_1^{recon.}$", linestyle="-", alpha=0.8, lw=2)
ax0.set_ylim(-20.0, 20.0)
ax0.legend(loc="upper right")
ax0.set_xlabel("System loops")
ax0.set_ylabel("Components of the system")
ax0.set_title(r"$x_1^{true}$ vs. $x_1^{recon.}$ from $z_1$ only");

In [ ]:
kwargs = dict(
    cmap="RdBu_r",
    vmin=-10,
    vmax=10,
)
fig, axs = plt.subplots(ncols=3, figsize=(12, 5), sharex=True, sharey=True)
axs = axs.flatten()
mpl = axs[0].scatter(x_t[:, 1], x_t[:, 2], c=x_t[:, 0], **kwargs)
axs[0].set_title(r"$x_1^{true}$")
cbar = fig.colorbar(mappable=mpl, ax=axs[0])
mpl = axs[1].scatter(x_t[:, 1], x_t[:, 2], c=x1_restored, **kwargs)
axs[1].set_title(r"$x_1^{recon}$")
fig.colorbar(mappable=mpl, ax=axs[1])
mpl = axs[2].scatter(
    x_t[:, 1], x_t[:, 2], c=(x_t[:, 0] - x1_restored) / x_t[:, 0], **kwargs
)
fig.colorbar(mappable=mpl, ax=axs[2])
axs[2].set_title(r"$(x_1^{true} - x_1^{recon})/x_1^{true}$")
for ax in axs:
    ax.set_xlabel(r"$x_2^{true}$")
    ax.set_ylabel(r"$x_3^{true}$")
fig.tight_layout()

In [ ]:
def calculate_x1_full(x2, x3, z1, z2, a, b, sigma, beta, rho):
    """Calculate the missing component x1 using
    - x2 of the Kalman_SEM
    - x3 of the Kalman_SEM
    - z1 of the Kalman_SEM
    - z2 of the Kalman_SEM
    - a containing the coefficients a1, a2, a3 (a1 is not used)
    - b containing the coefficients b1, b2, b3
    For a and b refer to eq.6(a+b) in the paper Tandeo et. al 2023.

    Parameters
    ----------
    x2: np.ndarray
        x2 of the Lorenz model.
        Needs to be dimension 1 with length N.
    x3: np.ndarray
        x3 of the Lorenz model.
        Needs to be dimension 1 with length N.
    z1: np.ndarray
        1st additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    z2: np.ndarray
        2nd additional component used for the Kalman_SEM().
        Needs to be dimension 1 with length N.
    a: np.ndarray
        Array containing the coefficients [a2,a3] for the equation:
        z1 = a2* d/dt(x2) + a3* d/dt(x3)
    b: np.ndarray
        Array containing the coefficients [b1, b2, b3] for the equation:
        z2 = b1* d/dt(z1) + b2* d2/dt2(x3) + b3* d2/dt2(x3)

    Returns
    -------
    np.ndarray
        The estimation of x1 based the coefficients provided and the assumption of knowing
        d/dt(x2) = ... and
        d/dt(x3) = ...
        from the Lorenz Model PDE.

    """
    a2 = a[0]
    a3 = a[1]
    b1 = b[0]
    b2 = b[1]
    b3 = b[2]

    coef_x1_square = (rho - x3) * (b2 * a3) + -b1 * a2 * x2
    coef_x1 = (
        (rho - x3) * (-(sigma + 1) * (b1 * a2) + b2 - a2)
        + x3 * (b1 * a2 * beta)
        + x2 * (-(sigma + beta + 1) * (b2 * a3) + b3 - a3)
    )
    coef_const = x2 * ((b1 + 1) * a2 - b2) + x3 * (a3 * beta) + -z1 + z2
    # calculate the nominator

    return np.array([coef_x1_square, coef_x1, coef_const])


coefs = calculate_x1_full(
    x2=x_s_V2[:, 0],
    x3=x_s_V2[:, 1],
    z1=x_s_V2[:, 2],
    z2=x_s_V2[:, 3],
    a=regress_a_V2_coef,
    b=regress_b_coef,
    sigma=sigma,
    beta=beta,
    rho=rho,
)

coefs = coefs / coefs[0]
coefs[np.abs(coefs) > 100] = np.nan
x1_pos = coefs[1] / 2 + np.sqrt((coefs[1] / 2) ** 2 - coefs[2])
x1_neg = coefs[1] / 2 - np.sqrt((coefs[1] / 2) ** 2 - coefs[2])